<a href="https://colab.research.google.com/github/kimberlyhau/StockMarketPredictor/blob/main/LSTMModel_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [4]:
import re

In [5]:
import os
os.chdir("drive/My Drive")

In [6]:
sentiment_file = os.path.join('/content/drive/My Drive/sentiments.txt')
stock_prices_folder = os.path.join('/content/drive/My Drive/stock_market_data')

In [8]:
with open(sentiment_file, 'r') as file_obj:
    text = file_obj.read()
    txt_list = text.split('[')

In [14]:
stock_data_lst = []
for filename in os.listdir(stock_prices_folder):
    if ".txt" in filename:
      #file_object = open(folder + "/" + filename, "r")
      with open(stock_prices_folder + "/" + filename, 'r') as file_obj:
        text = file_obj.read()
        stock_data_lst.append([filename.replace('.txt', ''), re.split(',|\n', text)])

In [18]:
stock_data_lst[0][1]

['Date',
 'Close Prices Real Estate',
 '2019-01-02',
 '377.6545187902822',
 '2019-01-03',
 '378.6804341088255',
 '2019-01-04',
 '382.3176520847931',
 '2019-01-07',
 '384.2787421698025',
 '2019-01-08',
 '390.98490886682697',
 '2019-01-09',
 '388.23277794595646',
 '2019-01-10',
 '395.632918319235',
 '2019-01-11',
 '396.4174421271933',
 '2019-01-14',
 '396.64635468752914',
 '2019-01-15',
 '401.5061864569485',
 '2019-01-16',
 '403.0354925700603',
 '2019-01-17',
 '404.59816922603846',
 '2019-01-18',
 '403.88945732691394',
 '2019-01-22',
 '402.93075964524706',
 '2019-01-23',
 '402.7758401816464',
 '2019-01-24',
 '404.86439106138783',
 '2019-01-25',
 '408.30952186086',
 '2019-01-28',
 '412.83843728151646',
 '2019-01-29',
 '416.23512950995877',
 '2019-01-30',
 '419.2616172932804',
 '2019-01-31',
 '424.99982535258425',
 '2019-02-01',
 '422.60186537142715',
 '2019-02-04',
 '424.6248421569101',
 '2019-02-05',
 '426.8768748841932',
 '2019-02-06',
 '422.9731826697953',
 '2019-02-07',
 '426.34018486

In [16]:
stock_data_lst

[['Real EstateSP500Data',
  ['Date',
   'Close Prices Real Estate',
   '2019-01-02',
   '377.6545187902822',
   '2019-01-03',
   '378.6804341088255',
   '2019-01-04',
   '382.3176520847931',
   '2019-01-07',
   '384.2787421698025',
   '2019-01-08',
   '390.98490886682697',
   '2019-01-09',
   '388.23277794595646',
   '2019-01-10',
   '395.632918319235',
   '2019-01-11',
   '396.4174421271933',
   '2019-01-14',
   '396.64635468752914',
   '2019-01-15',
   '401.5061864569485',
   '2019-01-16',
   '403.0354925700603',
   '2019-01-17',
   '404.59816922603846',
   '2019-01-18',
   '403.88945732691394',
   '2019-01-22',
   '402.93075964524706',
   '2019-01-23',
   '402.7758401816464',
   '2019-01-24',
   '404.86439106138783',
   '2019-01-25',
   '408.30952186086',
   '2019-01-28',
   '412.83843728151646',
   '2019-01-29',
   '416.23512950995877',
   '2019-01-30',
   '419.2616172932804',
   '2019-01-31',
   '424.99982535258425',
   '2019-02-01',
   '422.60186537142715',
   '2019-02-04',
   '4

In [12]:
stock_data_lst[-2]

['Date',
 'Close Prices Financials',
 '2019-01-02',
 '459.27398798313175',
 '2019-01-03',
 '450.1350767553301',
 '2019-01-04',
 '465.8768132528759',
 '2019-01-07',
 '466.6852552234171',
 '2019-01-08',
 '468.00622647636396',
 '2019-01-09',
 '469.6859430298252',
 '2019-01-10',
 '470.8899026034434',
 '2019-01-11',
 '471.0086363715951',
 '2019-01-14',
 '473.0702444650758',
 '2019-01-15',
 '478.1673356449385',
 '2019-01-16',
 '489.3438613087296',
 '2019-01-17',
 '491.4751936988996',
 '2019-01-18',
 '499.27882689259246',
 '2019-01-22',
 '493.6714659955313',
 '2019-01-23',
 '493.0003495143365',
 '2019-01-24',
 '493.26211388112273',
 '2019-01-25',
 '497.59835780786057',
 '2019-01-28',
 '496.0221766457093',
 '2019-01-29',
 '494.5542980324531',
 '2019-01-30',
 '496.72361396677707',
 '2019-01-31',
 '498.3421511801652',
 '2019-02-01',
 '501.72291359244394',
 '2019-02-04',
 '503.58957349361015',
 '2019-02-05',
 '503.33585292736063',
 '2019-02-06',
 '501.621741808555',
 '2019-02-07',
 '498.823468540

In [25]:
#stocks_df = pd.DataFrame(columns = ['Stock Name', 'Stock Price', 'Datestamp'])
stock_data = []
for stocks in stock_data_lst:
  #stocks_df = pd.DataFrame(columns = ['Stock Price', 'Datestamp'])
  stock_name = stocks[0]
  datestamp_lst = []
  stock_price_lst = []
  if stock_name != "SP500Data":
    for stock_elem in stocks[1]:
      if re.search('[a-zA-Z]', stock_elem):
        continue
      elif '-' in stock_elem:
        elems = stock_elem.split('-')
        datestamp = int(elems[0])*10000 + int(elems[1])*100 + int(elems[2])
        datestamp_lst.append(datestamp)
      elif '.' in stock_elem:
        stock_price = float(stock_elem)
        stock_price_lst.append(stock_price)
    stock_data.append([stock_name, pd.DataFrame(list(zip(stock_price_lst, datestamp_lst)), columns =['Stock Price', 'Datestamp'])])

In [26]:
len(stock_data)

11

In [19]:
datestamp_lst = []
sentiment_lst = []
for i in range(1, len(txt_list)):
  elems = re.split("'|-|T|:|Z|,|]| ", txt_list[i])
  while("" in elems) :
    elems.remove("")
  datestamp = int(elems[0])*10000000000 + int(elems[1])*100000000 + int(elems[2])*1000000 + int(elems[3])*10000 + int(elems[4])*100 + int(elems[5])
  sentiment = float(elems[6]) 
  datestamp_lst.append(datestamp)
  sentiment_lst.append(sentiment)
sentiment_df = pd.DataFrame(list(zip(sentiment_lst, datestamp_lst)), columns =['Sentiment', 'Datestamp'])

In [ ]:
sentiment_df_sorted = sentiment_df.sort_values('Datestamp')

In [ ]:
X_train = (sentiment_df_sorted[:int(sentiment_df.shape[0]*0.8)]).sample(frac=1).reset_index(drop=True)
test_and_val = (sentiment_df_sorted[int(sentiment_df.shape[0]*0.8):]).sample(frac=1)
X_test = test_and_val[:int(test_and_val.shape[0]*0.5)].reset_index(drop=True)
X_val = test_and_val[int(test_and_val.shape[0]*0.5):].reset_index(drop=True)

In [ ]:
y_train

In [ ]:
input_features = 3
hidden = 20
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.lstm = nn.LSTM (input_size=input_features, hidden_size= hidden, num_layers = 1,)
        self.linear = nn.Linear(hidden,1)

    def forward(self,x):
      out = self.lstm(x)
      out = self.linear(out)
      return out

In [ ]:
net = Model()
learning_rate = 0.05
criterion = nn.MSELoss()# BCE??
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
epochs = 5

for epoch in range(epochs):
  y = net(input)
  loss = criterion (y, target)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()